In [1]:
from PIL import Image, ImageDraw
import os

In [6]:
import os
import random
from PIL import Image, ImageDraw, ImageEnhance, ImageOps, ImageFilter

# config
OUTPUT_ROOT = "./dataset_final"
os.makedirs(OUTPUT_ROOT, exist_ok=True)

SIZES = [48, 128, 256]  # diferentes resoluciones
CARITAS_POR_CLASE = 50  # cantidad base por expresión
AUGMENTATIONS = 4       # cuántas versiones extra por imagen

# generador de caritas base
def crear_carita(expresion, size):
    # Fondo aleatorio (colores suaves)
    fondo_color = random.choice([
        (255, 255, 255), (240, 240, 255), (255, 240, 230),
        (250, 255, 240), (245, 245, 245), (255, 255, 200),
        (230, 250, 230), (230, 240, 250), (250, 230, 250)
    ])
    img = Image.new("RGB", (size, size), fondo_color)
    draw = ImageDraw.Draw(img)

    # Color de la cara
    color_cara = random.choice([
        (255, 230, 0),   # amarillo clásico
        (255, 220, 100), # dorado
        (255, 180, 100), # naranja suave
        (255, 250, 150), # crema claro
        (250, 200, 150), # beige
        (255, 200, 200), # rosado suave
        (220, 220, 220), # gris claro
        (255, 255, 255), # blanco
        (200, 230, 255), # azul claro
        (200, 255, 200)  # verde menta
    ])

    # Decidir si tendrá borde
    borde = random.choice([True, False])
    outline_color = (0, 0, 0) if borde else color_cara

    margen = size // 12
    draw.ellipse((margen, margen, size - margen, size - margen),
                 fill=color_cara, outline=outline_color)

    ojo_y = size // 2 - size // 5
    grosor = random.choice([1, 2, 3])

    # Dibujar según expresión
    if expresion != "muerto":
        draw.ellipse((size*0.3, ojo_y, size*0.38, ojo_y + size*0.08), fill=(0, 0, 0))
        draw.ellipse((size*0.62, ojo_y, size*0.70, ojo_y + size*0.08), fill=(0, 0, 0))

    if expresion == "feliz":
        draw.arc((size*0.3, size*0.46, size*0.70, size*0.75),
                 start=0, end=180, fill=(0, 0, 0), width=grosor)

    elif expresion == "triste":
        draw.arc((size*0.3, size*0.55, size*0.70, size*0.85),
                 start=180, end=360, fill=(0, 0, 0), width=grosor)

    elif expresion == "enojado":
        draw.line((size*0.25, size*0.25, size*0.40, size*0.32),
                  fill=(0, 0, 0), width=2)
        draw.line((size*0.60, size*0.32, size*0.75, size*0.25),
                  fill=(0, 0, 0), width=2)
        draw.ellipse((size*0.3, ojo_y + size*0.05, size*0.38, ojo_y + size*0.12), fill=(0, 0, 0))
        draw.ellipse((size*0.62, ojo_y + size*0.05, size*0.70, ojo_y + size*0.12), fill=(0, 0, 0))
        draw.arc((size*0.3, size*0.55, size*0.70, size*0.85),
                 start=180, end=360, fill=(0, 0, 0), width=2)

    elif expresion == "sorprendida":
        draw.ellipse((size*0.45, size*0.60, size*0.55, size*0.70), fill=(0, 0, 0))

    elif expresion == "neutral":
        draw.line((size*0.35, size*0.65, size*0.65, size*0.65),
                  fill=(0, 0, 0), width=grosor)

    elif expresion == "muerto":
        for (x1, y1, x2, y2) in [
            (size*0.3, size*0.38, size*0.38, size*0.46),
            (size*0.38, size*0.38, size*0.3, size*0.46),
            (size*0.62, size*0.38, size*0.70, size*0.46),
            (size*0.70, size*0.38, size*0.62, size*0.46)
        ]:
            draw.line((x1, y1, x2, y2), fill=(0, 0, 0), width=1)
        draw.arc((size*0.3, size*0.55, size*0.70, size*0.85),
                 start=180, end=360, fill=(0, 0, 0), width=2)

    # elif expresion == "nervioso":
    #     draw.rectangle((size*0.33, size*0.63, size*0.67, size*0.74),
    #                    outline=(0, 0, 0), fill=(255, 255, 255))
    #     for x in [size*0.37, size*0.45, size*0.53, size*0.61]:
    #         draw.line((x, size*0.63, x, size*0.74), fill=(0, 0, 0))
    #     draw.line((size*0.33, size*0.69, size*0.67, size*0.69), fill=(0, 0, 0))

    return img


# aumentaciones
def augment_image(img):
    aug = img.copy()
    aug = ImageEnhance.Color(aug).enhance(random.uniform(0.6, 1.6))
    aug = ImageEnhance.Brightness(aug).enhance(random.uniform(0.8, 1.3))
    aug = ImageEnhance.Contrast(aug).enhance(random.uniform(0.8, 1.3))

    if random.random() > 0.5:
        aug = ImageOps.mirror(aug)
    aug = aug.rotate(random.randint(-15, 15))

    if random.random() > 0.5:
        w, h = aug.size
        crop_size = random.randint(int(w * 0.8), w)
        left = random.randint(0, w - crop_size)
        top = random.randint(0, h - crop_size)
        aug = aug.crop((left, top, left + crop_size, top + crop_size)).resize((w, h))

    if random.random() > 0.7:
        aug = aug.filter(ImageFilter.GaussianBlur(radius=random.uniform(0.3, 1.0)))

    return aug


# gernerar por tamanio y clase
expresiones = ["feliz", "triste", "enojado", "sorprendida", "neutral", "muerto"]

for size in SIZES:
    print(f"\nGenerando caritas {size}x{size}...")
    for expresion in expresiones:
        # out_dir = os.path.join(OUTPUT_ROOT, f"{size}x{size}", expresion)
        out_dir = os.path.join(OUTPUT_ROOT, expresion)

        os.makedirs(out_dir, exist_ok=True)

        for i in range(CARITAS_POR_CLASE):
            base = crear_carita(expresion, size)
            base.save(os.path.join(out_dir, f"{expresion}_{i:03d}.png"))

            # Generar aumentaciones
            for j in range(AUGMENTATIONS):
                aug = augment_image(base)
                aug.save(os.path.join(out_dir, f"{expresion}_{i:03d}_aug{j}.png"))

        print(f"{expresion}: {CARITAS_POR_CLASE * (1 + AUGMENTATIONS)} imágenes ({size}px) generadas.")

print("\nDataset generado en './dataset_final/'")



Generando caritas 48x48...
feliz: 250 imágenes (48px) generadas.
triste: 250 imágenes (48px) generadas.
enojado: 250 imágenes (48px) generadas.
sorprendida: 250 imágenes (48px) generadas.
neutral: 250 imágenes (48px) generadas.
muerto: 250 imágenes (48px) generadas.

Generando caritas 128x128...
feliz: 250 imágenes (128px) generadas.
triste: 250 imágenes (128px) generadas.
enojado: 250 imágenes (128px) generadas.
sorprendida: 250 imágenes (128px) generadas.
neutral: 250 imágenes (128px) generadas.
muerto: 250 imágenes (128px) generadas.

Generando caritas 256x256...
feliz: 250 imágenes (256px) generadas.
triste: 250 imágenes (256px) generadas.
enojado: 250 imágenes (256px) generadas.
sorprendida: 250 imágenes (256px) generadas.
neutral: 250 imágenes (256px) generadas.
muerto: 250 imágenes (256px) generadas.

Dataset generado en './dataset_final/'


In [ ]:
import os
import random
from PIL import Image, ImageEnhance, ImageOps, ImageFilter

#  CONFIG
INPUT_ROOT = "./faces"                 # Dataset original
OUTPUT_BASE = "./dataset_final"        # Carpeta raíz de salida
AUGMENTATIONS = 5                          # Cuántas versiones extra por imagen
SIZES = [48, 128, 256]                     # Diferentes resoluciones de salida

#  FUNCIÓN DE AUMENTACIÓN 
def augment_image(img):
    aug = img.copy().convert("RGB")

    # Variaciones de color, brillo, contraste 
    aug = ImageEnhance.Color(aug).enhance(random.uniform(0.6, 1.6))
    aug = ImageEnhance.Brightness(aug).enhance(random.uniform(0.8, 1.3))
    aug = ImageEnhance.Contrast(aug).enhance(random.uniform(0.8, 1.3))

    # Rotación y espejo 
    if random.random() > 0.5:
        aug = ImageOps.mirror(aug)
    aug = aug.rotate(random.randint(-15, 15), expand=True)

    # Zoom o recorte 
    if random.random() > 0.5:
        w, h = aug.size
        crop_size = random.randint(int(w * 0.8), w)
        left = random.randint(0, max(0, w - crop_size))
        top = random.randint(0, max(0, h - crop_size))
        aug = aug.crop((left, top, left + crop_size, top + crop_size)).resize((w, h))

    # Desenfoque leve (simula ruido o pérdida de enfoque)
    if random.random() > 0.7:
        aug = aug.filter(ImageFilter.GaussianBlur(radius=random.uniform(0.3, 1.0)))

    return aug


# PROCESAMIENTO 
for size in SIZES:
    TARGET_SIZE = (size, size)
    # OUTPUT_ROOT = os.path.join(OUTPUT_BASE, f"{size}x{size}")
    os.makedirs(OUTPUT_ROOT, exist_ok=True)

    print(f"\nGenerando dataset aumentado de {size}x{size} píxeles...")

    for folder in os.listdir(INPUT_ROOT):
        input_path = os.path.join(INPUT_ROOT, folder)
        if not os.path.isdir(input_path):
            continue

        output_path = os.path.join(OUTPUT_ROOT, folder)
        os.makedirs(output_path, exist_ok=True)

        print(f"Procesando carpeta: {folder}")

        for filename in os.listdir(input_path):
            if not filename.lower().endswith((".png", ".jpg", ".jpeg")):
                continue

            img_path = os.path.join(input_path, filename)
            try:
                img = Image.open(img_path).convert("RGB").resize(TARGET_SIZE)

                # Guardar imagen base redimensionada
                base_name = os.path.splitext(filename)[0]
                img.save(os.path.join(output_path, f"{base_name}_base_{size}x{size}.png"))

                # Generar versiones aumentadas
                for i in range(AUGMENTATIONS):
                    aug = augment_image(img)
                    aug.save(os.path.join(output_path, f"{base_name}_aug{i}_{size}x{size}.png"))

            except Exception as e:
                print(f"Error procesando {filename}: {e}")

        print(f"Carpeta '{folder}' aumentada y guardada en '{output_path}'")

print("\nDataset generado en './archive_augmented/'")



Generando dataset aumentado de 48x48 píxeles...
Procesando carpeta: confundido
Carpeta 'confundido' aumentada y guardada en './dataset_final\confundido'
Procesando carpeta: enojado


c:\Users\angel\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Carpeta 'enojado' aumentada y guardada en './dataset_final\enojado'
Procesando carpeta: feliz
Carpeta 'feliz' aumentada y guardada en './dataset_final\feliz'
Procesando carpeta: muerto
Carpeta 'muerto' aumentada y guardada en './dataset_final\muerto'
Procesando carpeta: neutral
Carpeta 'neutral' aumentada y guardada en './dataset_final\neutral'
Procesando carpeta: sorprendida
Carpeta 'sorprendida' aumentada y guardada en './dataset_final\sorprendida'
Procesando carpeta: triste
Carpeta 'triste' aumentada y guardada en './dataset_final\triste'

Generando dataset aumentado de 128x128 píxeles...
Procesando carpeta: confundido
Carpeta 'confundido' aumentada y guardada en './dataset_final\confundido'
Procesando carpeta: enojado
Carpeta 'enojado' aumentada y guardada en './dataset_final\enojado'
Procesando carpeta: feliz
Carpeta 'feliz' aumentada y guardada en './dataset_final\feliz'
Procesando carpeta: muerto
Carpeta 'muerto' aumentada y guardada en './dataset_final\muerto'
Procesando carpet

In [8]:
import os

DATASET_DIR = "./dataset_final"

# contar imágenes totales y por clase
total = 0
counts = {}

for clase in os.listdir(DATASET_DIR):
    path = os.path.join(DATASET_DIR, clase)
    if not os.path.isdir(path):
        continue

    count = len([f for f in os.listdir(path) if f.endswith((".png", ".jpg", ".jpeg"))])
    counts[clase] = count
    total += count

print("Total de imágenes:", total)
for clase, c in counts.items():
    print(f"  • {clase}: {c} imágenes")


Total de imágenes: 7062
  • confundido: 396 imágenes
  • enojado: 934 imágenes
  • feliz: 1258 imágenes
  • muerto: 790 imágenes
  • neutral: 718 imágenes
  • sorprendida: 1456 imágenes
  • triste: 1510 imágenes


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_gen = ImageDataGenerator(rescale=1/255, validation_split=0.2)

train_data = data_gen.flow_from_directory(
    "./dataset_final",
    target_size=(128, 128),
    batch_size=32,
    class_mode="categorical",
    subset="training"
)

val_data = data_gen.flow_from_directory(
    "./dataset_final",
    target_size=(128, 128),
    batch_size=32,
    class_mode="categorical",
    subset="validation"
)

print("Total de imágenes de entrenamiento:", train_data.samples)
print("Total de imágenes de validación:", val_data.samples)




Found 5652 images belonging to 7 classes.
Found 1410 images belonging to 7 classes.
Total de imágenes de entrenamiento: 5652
Total de imágenes de validación: 1410
